# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import json


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mar del Plata,-38.0023,-57.5575,57.15,54,0,17.27,AR,1698699460
1,1,Upata,8.0086,-62.3989,79.86,73,100,3.89,VE,1698699460
2,2,Kozan,37.4552,35.8157,73.02,35,100,5.28,TR,1698699460
3,3,Squamish,49.7014,-123.1512,56.21,44,31,2.30,CA,1698699460
4,4,Valparaíso,-33.0393,-71.6273,62.02,65,0,13.80,CL,1698699461


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    #frame_width = 700,
    #frame_height = 500,
    size = "Humidity",
    #scale = 0.01,
    color = "City"
)


# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow = city_data_df.loc[(city_data_df["Max Temp"]<65) & (city_data_df["Max Temp"]>21) & 
                                    (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0) ,:]

# Drop any rows with null values
city_data_narrow = city_data_narrow.dropna()

# Display sample data
city_data_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
176,176,Psachná,38.5833,23.6333,63.64,73,0,0.29,GR,1698699488
249,249,Trikala,39.5553,21.7675,64.76,60,0,1.61,GR,1698699500
300,300,Spring Creek,40.7266,-115.5859,44.19,19,0,3.47,US,1698699509
325,325,Carqueiranne,43.0953,6.0730,61.50,85,0,3.44,FR,1698699513
493,493,Tadotsu,34.2750,133.7500,57.97,64,0,2.98,JP,1698699540
538,538,Akureyri,65.6835,-18.0878,21.13,93,0,0.00,IS,1698699547
568,568,Kurayoshi,35.4333,133.8167,53.62,75,0,4.00,JP,1698699554


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_narrow[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\simon\AppData\Local\Temp\ipykernel_29808\508403183.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
176,Psachná,GR,38.5833,23.6333,73,
249,Trikala,GR,39.5553,21.7675,60,
300,Spring Creek,US,40.7266,-115.5859,19,
325,Carqueiranne,FR,43.0953,6.0730,85,
493,Tadotsu,JP,34.2750,133.7500,64,
538,Akureyri,IS,65.6835,-18.0878,93,
568,Kurayoshi,JP,35.4333,133.8167,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lati = hotel_df.loc[index,"Lat"]
    long = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lati},{radius}"
    params["bias"] = f"proximity:{long},{lati}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Psachná - nearest hotel: ΕυβοΪκή Ακτή
Trikala - nearest hotel: Αχίλλειον
Spring Creek - nearest hotel: No hotel found
Carqueiranne - nearest hotel: Le Richiardi
Tadotsu - nearest hotel: 香露軒
Akureyri - nearest hotel: Hótel Norðurland
Kurayoshi - nearest hotel: 倉吉シティホテル


,City,Country,Lat,Lng,Humidity,Hotel Name
176,Psachná,GR,38.5833,23.6333,73,ΕυβοΪκή Ακτή
249,Trikala,GR,39.5553,21.7675,60,Αχίλλειον
300,Spring Creek,US,40.7266,-115.5859,19,No hotel found
325,Carqueiranne,FR,43.0953,6.0730,85,Le Richiardi
493,Tadotsu,JP,34.2750,133.7500,64,香露軒
538,Akureyri,IS,65.6835,-18.0878,93,Hótel Norðurland
568,Kurayoshi,JP,35.4333,133.8167,75,倉吉シティホテル


{'error': 'Bad Request',
 'message': '"filter[0]" does not match any of the allowed types',
 'statusCode': 400}


{'apiKey': 'a60b8f746db1448aa5b5b2a07af9e93b',
 'filter': 'circle:176     23.6333\n249     21.7675\n300   -115.5859\n325      6.0730\n493    133.7500\n538    -18.0878\n568    133.8167\nName: Lng, dtype: float64,176    38.5833\n249    39.5553\n300    40.7266\n325    43.0953\n493    34.2750\n538    65.6835\n568    35.4333\nName: Lat, dtype: float64,10000',
 'bias': 'proximity:176     23.6333\n249     21.7675\n300   -115.5859\n325      6.0730\n493    133.7500\n538    -18.0878\n568    133.8167\nName: Lng, dtype: float64,176    38.5833\n249    39.5553\n300    40.7266\n325    43.0953\n493    34.2750\n538    65.6835\n568    35.4333\nName: Lat, dtype: float64'}

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)